<a href="https://colab.research.google.com/github/HeshanDissanayake/SIIM_ISIC_melanoma_classification/blob/main/SIIM_ISIC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!curl 'https://storage.googleapis.com/kaggle-data-sets/687793%2F1205903%2Fbundle%2Farchive.zip?GoogleAccessId=gcp-kaggle-com@kaggle-161607.iam.gserviceaccount.com&Expires=1592876955&Signature=e1XfUeSR1WAk0NUINE6lVoZl4aRmew0bEvGhm5dO%2F2gJMOSDBBjMuY5W%2BD2Dn2lnVUJuVatrP27LmbKQOKKZkARn2eyEYV0nyIU5dnOjT9yBymaHDRY9NifpOb%2BcRoPD%2BqIprjXA2rowtClLEr8znTvLw458pQyJFUpZLQLBdEu6gyxVwJrCsXr4VG6oBfKyqI17oVYOgOVhvAvskrDQqCAQGmp6AcqHhEPXWWUXFiGSzexx28OCSEbqzsx2Q4a2nk6BMxrl%2BReAIu2O%2FzRbn7rZqmLasVeyO%2F5c3Xfpj6nhydW4ZdeK2x2RSPlw4JrvZ1UgqdVIbNzP9ZL7EyOTbw%3D%3D' -H 'User-Agent: Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:77.0) Gecko/20100101 Firefox/77.0' -H 'Accept: text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8' -H 'Accept-Language: en-US,en;q=0.5' --compressed -H 'Referer: https://www.kaggle.com/' -H 'Connection: keep-alive' -H 'Cookie: cuntwars_user_id=xXNhMHwAmW; _ga=GA1.3.1485321816.1568920993' -H 'Upgrade-Insecure-Requests: 1' -H 'TE: Trailers' -o cancer.zip

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 2021M  100 2021M    0     0  81.4M      0  0:00:24  0:00:24 --:--:-- 93.0M


In [ ]:
!unzip cancer.zip

In [ ]:
import os
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import random
import cv2
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from shutil import copy
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, add, Dropout, Flatten, Dense, Reshape
from tensorflow.keras.models import Model
from zipfile import ZipFile
import sys
from tensorflow.keras.models import Sequential, load_model
from  tensorflow import keras

In [ ]:
allData = pd.read_csv('/content/train.csv', sep = ',')
df = pd.DataFrame(allData)
malignant = df.groupby(df.target).get_group(1) 
benign = df.groupby(df.target).get_group(0)

malignant_file_names = malignant.image_name.values
benign_file_names = benign.image_name.values

In [ ]:
len(malignant_file_names)


584

In [ ]:
!rm -R /content/Data

In [ ]:

!mkdir Data /content/Data/maignant /content/Data/maignant/train  
!mkdir /content/Data/train /content/Data/test 
!mkdir /content/Data/train/benign /content/Data/train/malignant /content/Data/test/benign /content/Data/test/malignant

In [ ]:
#Copy benigns
source = '/content/640x640/train'
traindest = '/content/Data/train/benign' 
testdest = '/content/Data/test/benign' 
trainSize = 1500
testSize = 1000
benign_file_names_train = benign_file_names[:trainSize]
benign_file_names_test = benign_file_names[trainSize:trainSize+testSize]

for i,file in enumerate(benign_file_names_train):
  os.system('cp -r %s %s'%(source+'/'+file+'.jpg', traindest+'/'+file+'.jpg'))
  sys.stdout.write('\r %d%%: Copying %s to %s'%((i/len(benign_file_names_train))*100, file, traindest))
  sys.stdout.flush()
print("\rComplete")

# for i,file in enumerate(benign_file_names_test):
#   os.system('cp -r %s %s'%(source+'/'+file+'.jpg', testdest+'/'+file+'.jpg'))
#   sys.stdout.write('\r %d%%: Copying %s to %s'%((i/len(benign_file_names_test))*100, file, testdest))
#   sys.stdout.flush()
# print("\rComplete")

Complete


In [ ]:
#Copy malignants
source = '/content/640x640/train'
dest = '/content/Data/train/malignant' 
for i,file in enumerate(malignant_file_names):
  os.system('cp -r %s %s'%(source+'/'+file+'.jpg', dest+'/'+file+'.jpg'))  
  sys.stdout.write('\r %d%%: Copying %s to %s'%((i/len(malignant_file_names))*100, file, dest))
  sys.stdout.flush()
print("\rComplete")

Complete


In [ ]:
#datagen for the malignant augmentation
datagen = ImageDataGenerator(rotation_range=1,
                              zoom_range = 0.5,
                              fill_mode="nearest",
                              horizontal_flip=True,
                              vertical_flip=True)

In [ ]:
images = datagen.flow_from_directory( directory='/content/Data/maignant',
                                target_size=(640, 640),
                                color_mode="rgb",
                                shuffle=True,
                                save_to_dir='/content/Data/train/malignant',
                                save_format="jpg",
                                seed = 1,
                                batch_size = 1
                                )


Found 584 images belonging to 1 classes.


In [ ]:
tot = 0
for i, batch in enumerate(images):
  if i > 7000:
    break
  tot = tot + batch[0].shape[0]
  sys.stdout.write('\r Total number of augmented images = %d'%tot)
  sys.stdout.flush()

 Total number of augmented images = 7001

In [ ]:
testdatagen = ImageDataGenerator(rotation_range=0.6,
                              zoom_range = 0.2,
                              fill_mode="nearest",
                              horizontal_flip=True,
                              vertical_flip=True)
images = testdatagen.flow_from_directory( directory='/content/Data/maignant',
                                target_size=(640, 640),
                                color_mode="rgb",
                                shuffle=True,
                                save_to_dir='/content/Data/test/malignant',
                                save_format="jpg",
                                seed = 1,
                                batch_size = 1
                                )


Found 584 images belonging to 1 classes.


In [ ]:
tot = 0
for i, batch in enumerate(images):
  if i > 1000:
    break
  tot = tot + batch[0].shape[0]
  sys.stdout.write('\r Total number of augmented images = %d'%tot)
  sys.stdout.flush()

 Total number of augmented images = 1001

In [ ]:
trainDatagen = ImageDataGenerator(rescale=1./255,
                                  rotation_range=0.6,
                                  zoom_range = 0.3,
                                  fill_mode="nearest",
                                  horizontal_flip=True,
                                  vertical_flip=True)
testDatagen = ImageDataGenerator(rescale=1./255)
traingen = trainDatagen.flow_from_directory(directory ='/content/Data/train',
                                       class_mode = 'binary',
                                       target_size = (320, 320),batch_size = 50)
testgen = testDatagen.flow_from_directory(directory ='/content/Data/test',
                                       class_mode = 'binary',
                                       target_size = (224, 224),batch_size = 20)

Found 2084 images belonging to 2 classes.
Found 0 images belonging to 2 classes.


In [ ]:
input = Input(shape = (320, 320, 3))
l1 = Conv2D(16, (3,3), activation = 'relu', padding = 'same', kernel_regularizer= keras.regularizers.l2(1e-4))(input) 
l1 = Dropout(0.1)(l1)
l2 = MaxPooling2D(2,2)(l1)

l3 = Conv2D(32, (3,3), activation = 'relu', padding = 'same', kernel_regularizer= keras.regularizers.l2(1e-4))(l2) 
l4 = MaxPooling2D(2,2)(l3)

l5 = Conv2D(64, (3,3), activation = 'relu', padding = 'same', kernel_regularizer= keras.regularizers.l2(1e-4))(l4)
l6 = MaxPooling2D(2,2)(l5)

l7 = Conv2D(128, (3,3), activation = 'relu', padding = 'same', kernel_regularizer= keras.regularizers.l2(1e-4))(l6) 
l8 = MaxPooling2D(2,2)(l7)

l9 = Conv2D(256, (3,3), activation = 'relu', padding = 'same', kernel_regularizer= keras.regularizers.l2(1e-4))(l8) 
l10 = Conv2D(512, (3,3), activation = 'relu', padding = 'same', kernel_regularizer= keras.regularizers.l2(1e-4))(l9) 
l11 = MaxPooling2D(2,2)(l10)

l12 = Conv2D(256, (3,3), activation = 'relu', padding = 'same', kernel_regularizer= keras.regularizers.l2(1e-4))(l11) 
l12 = Dropout(0.3)(l12)

l13 = MaxPooling2D(2,2)(l12)

l14 = Flatten()(l13)
l15 = Dense(100, activation = 'relu',kernel_regularizer= keras.regularizers.l2(1e-5))(l14)
l15 = Dropout(0.2)(l15)
l16 = Dense(100, activation = 'relu',kernel_regularizer= keras.regularizers.l2(1e-5))(l15)
l16 = Dropout(0.1)(l16)
l17 = Dense(1, activation = 'sigmoid')(l16)

net = Model(input, l17)
net.summary()

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 320, 320, 3)]     0         
_________________________________________________________________
conv2d_14 (Conv2D)           (None, 320, 320, 16)      448       
_________________________________________________________________
dropout_8 (Dropout)          (None, 320, 320, 16)      0         
_________________________________________________________________
max_pooling2d_12 (MaxPooling (None, 160, 160, 16)      0         
_________________________________________________________________
conv2d_15 (Conv2D)           (None, 160, 160, 32)      4640      
_________________________________________________________________
max_pooling2d_13 (MaxPooling (None, 80, 80, 32)        0         
_________________________________________________________________
conv2d_16 (Conv2D)           (None, 80, 80, 64)        1849

In [ ]:

class Callback(keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs=None):
      keys = list(logs.keys())
      if((epoch+1)%20 == 0):
        dir = '/content/drive/My Drive/cancer/models/v2/epoch_%d.h5'%epoch
        net.save(dir)

callback = Callback()

In [ ]:
net.compile(optimizer = 'adam', loss = "binary_crossentropy", metrics=['accuracy'])

In [ ]:
net.fit(traingen, steps_per_epoch=40, epochs= 80, callbacks=[callback])

In [ ]:
new = net.save('/content/drive/My Drive/cancer/v3.h5')

In [ ]:
net = load_model('/content/drive/My Drive/cancer/models/v1/v1_acc_8214.h5')

In [ ]:
output = []
for root,dir,files in os.walk('/content/640x640/test'):
  for i,image in enumerate(files):
    image_name = image.strip('.jpg')
    img =  plt.imread(root+'/'+image)
    img = cv2.resize(img,(320, 320)).reshape(1, 320, 320,3)
    prediction = modIncep.predict(img/255)
    
    print(int((i/len(files))*100),image_name, round(prediction[0][0],5))
    output.append([image_name,round(prediction[0][0],5)])
    # break


In [ ]:

df = pd.DataFrame(output)

In [ ]:
df.to_csv('kagglev21_incep.csv', index=False)

In [ ]:
#  print the propotion of 1s
a = np.array(df[1].values)
b = a > 0.5
print(np.sum(b)/a.shape[0])

0.18238936441449644


Re training on autoencoder


In [ ]:
encoder = load_model('/content/drive/My Drive/cancer/models/autoencoder/epoch_150.h5')
for layer in encoder.layers:
  layer.trainable = False

In [ ]:
encoder.summary()

In [ ]:
last_outs = encoder.get_layer('flatten').output
x1 = Dense(100, activation='relu', kernel_regularizer= keras.regularizers.l2(1e-5))(last_outs)
x1= Dropout(0.2)(x1)

x2 = Dense(200, activation='relu', kernel_regularizer= keras.regularizers.l2(1e-5))(x1)
x2= Dropout(0.2)(x2)

x3 = Dense(200, activation='relu', kernel_regularizer= keras.regularizers.l2(1e-5))(x2)
x3= Dropout(0.2)(x3)

outs = Dense(1, activation='sigmoid')(x3)
model = Model(encoder.input, outs)
model.summary()

In [ ]:
model.compile(optimizer = keras.optimizers.RMSprop(learning_rate=0.001) , loss = "binary_crossentropy", metrics=['accuracy'])

In [ ]:
model.fit(traingen, steps_per_epoch=70, epochs= 10)

In [ ]:
modIncep.evaluate(traingen)

31/42 [=====================>........] - ETA: 13s - loss: 0.0984 - accuracy: 0.9600

KeyboardInterrupt: ignored

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


Training on  VGG

In [ ]:
from tensorflow.keras.applications.inception_v3 import InceptionV3

incep = InceptionV3()

96116736/96112376 [==============================] - 2s 0us/step


In [ ]:
incep.summary()

In [ ]:
for layer in incep.layers:
  layer.trainable = False

break_layer_out = incep.get_layer('mixed8').output

In [ ]:
x2 = Flatten()(break_layer_out)

x2 = Dense(128, activation='relu')(x2)
x2= Dropout(0.3)(x2)

# x2 = Dense(256, activation='relu')(x2)
# x2= Dropout(0.1)(x2)

# x2 = Dense(256, activation='relu')(x2)
# x2= Dropout(0.1)(x2)

out = Dense(1, activation='sigmoid')(x2)

modIncep = Model(incep.input, out)
modIncep.summary()

In [ ]:
modIncep.compile(optimizer = 'adam', loss = "binary_crossentropy", metrics=['accuracy'])

In [ ]:
modIncep.fit(traingen, steps_per_epoch=40, epochs= 100)

In [ ]:
modIncep.save('/content/drive/My Drive/cancer/acc_94.h5')

In [ ]:
for i in range(3,100,4):
  print("{memory_array[10'd%d], memory_array[10'd%d], memory_array[10'd%d], memory_array[10'd%d]}"%(i,i-1,i-2,i-3))